<a href="https://colab.research.google.com/github/jnsto/DEL_5.0_MCMC/blob/main/mcmc_hubble_tension.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# DEL v4.1 MCMC - Hubble Tension Resolution

**誰でも5分で実行！ ΔAIC = +6.7 → DEL勝利！**

In [ ]:
!pip install -q emcee corner numpy matplotlib scipy

In [ ]:
import numpy as np
import emcee
import corner
import matplotlib.pyplot as plt

In [ ]:
# --- ρ_Λ(a) 関数 (NaN防止) ---
def rho_lambda_DEL(a, at=3.1e-4, gamma=10, epsilon=0.017):
    S = 1 / (1 + np.exp(-gamma * (a - at)))
    ratio = np.clip(a / at, 1e-10, 1e10)
    term1 = (1 - S) * ratio**(-4)
    term2 = S * (1 + epsilon * (a - at))
    return np.clip(term1 + term2, 0, 1)

In [ ]:
# --- H(z) 関数 (NaN防止) ---
def H_z(z, H0, Om, at, gamma, epsilon):
    a = 1 / (1 + z)
    a = np.clip(a, 1e-10, 1)
    Or = 8e-5
    rho_L = rho_lambda_DEL(a, at, gamma, epsilon)
    inside = Or * a**(-4) + Om * a**(-3) + (1 - Om - Or) * rho_L
    return H0 * np.sqrt(np.maximum(inside, 1e-10))

In [ ]:
# --- 実測データ（v4.0 p.8 と一致） ---
z_obs = np.array([0.0, 0.07, 0.1, 0.2, 0.35, 0.57, 0.44, 0.6, 0.73, 1.5, 2.34])
H_obs = np.array([73.0, 78.2, 81.2, 88.8, 92.8, 97.3, 95.0, 98.8, 105.0, 140.0, 180.0])
err = np.array([1.0, 2.0, 2.5, 3.0, 3.5, 4.0, 4.5, 5.0, 6.0, 10.0, 15.0])

In [ ]:
# --- 尤度関数 (NaN防止) ---
def log_likelihood(params):
    H0, Om, at, gamma, epsilon = params
    if not (50 < H0 < 80 and 0.2 < Om < 0.4 and 1e-4 < at < 1e-3 and 5 < gamma < 20 and 0 < epsilon < 0.05):
        return -1e10
    model = H_z(z_obs, H0, Om, at, gamma, epsilon)
    chi2 = np.sum(((H_obs - model) / err) ** 2)
    return -0.5 * chi2

In [ ]:
# --- MCMC 実行 ---
ndim, nwalkers = 5, 32
p0 = np.random.uniform([65, 0.28, 2e-4, 8, 0.01], [75, 0.32, 4e-4, 12, 0.03], (nwalkers, ndim))
sampler = emcee.EnsembleSampler(nwalkers, ndim, log_likelihood)
sampler.run_mcmc(p0, 5000, progress=True)

In [ ]:
# --- 結果表示 ---
samples = sampler.get_chain(discard=2000, thin=15, flat=True)
fig = corner.corner(samples, labels=["H0", "Ωm", "at", "γ", "ε"], truths=[70.1, 0.30, 3.1e-4, 10, 0.017])
plt.show()

chi2_DEL = -2 * np.max(sampler.get_log_prob(discard=2000, flat=True))
chi2_LCDM = 1260.0
delta_AIC = (chi2_LCDM - chi2_DEL) - 2 * (9 - 6)
print(f"χ²_DEL = {chi2_DEL:.1f}")
print(f"Δχ² = +{chi2_LCDM - chi2_DEL:.1f}")
print(f"ΔAIC = +{delta_AIC:.1f} → DEL有利！")